In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [2]:
class TrafficSignNet:
  @staticmethod
  def build(width, height, depth, classes):
    model= Sequential()
    inputShape = (height, width, depth)
    chanDim = -1
    model.add(Conv2D(8,(5,5), padding = "same", input_shape = inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(16, (3,3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(16, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
  
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    
    return model


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from skimage import transform
from skimage import exposure
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import os

In [4]:
def load_split(basepath, csvPath):
    data = []
    labels = []
    rows = open(csvPath).read().strip().split("\n")[1:]
    random.shuffle(rows)
    
    for(i, row) in enumerate(rows):
        if i> 0 and i % 1000 == 0:
            print("[INFO] processed {} total images".format(i))
        
        (label, imagePath) = row.strip().split(",")[-2:]
        
        imagePath = os.path.sep.join([basePath, imagePath])
        image - io.imread(imagePath)
        
        image = transform.resize(image, (32, 32))
        image = exposure.equalizer_adapthist(imahe, clip_limit = 0.1)
        
        data.append(image)
        labels.append(int(label))
    
    data = np.array(data)
    labels = np. array(labels)
    
    return (data, labels)


In [5]:
ap = argparse.ArgumentParser()

In [7]:
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True, help="path to input GTSRB")
ap.add_argument("-m", "--model", required=True,help="path to output model")
ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to training history plot")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -d DATASET -m MODEL [-p PLOT]
ipykernel_launcher.py: error: the following arguments are required: -d/--dataset, -m/--model


SystemExit: 2

In [7]:
NUM_EPOCHS = 30
INIT_LR = 1e-3
BS = 64

In [9]:
labelNames = open("Meta.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]


In [10]:
train.path = os.path.sep.join([args["dataset"], "Train.csv"])
test. path = os.path.sep.join([args["dataset"], "Test.csv"])

print("[INFO] loading training and test data...")
(trainX, trainY) = load_split(args["dataset"],  trainPath)
(testX, testY) = load_split(args["dataset"], testPath)

trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

numLabels = len(np.unique(trainY))
trainY = to_categorical(trainY, numLabels)
testY = to_categorical(testY, numLabels)

classTotals = trainY.sum(axis=0)
classWeight = dict()

for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

NameError: name 'args' is not defined

In [20]:
aug = ImageDataGenerator(rotation_range = 10, zoom_range = 0.15, width_shift_range = 0.1, shear_range = 0.15, horizontal_flip = False, vertical_flip = False, fill_mode = "nearest")

In [23]:
print("[INFO] compiling model...")
opt= Adam(lr = INIT_LR, decay= INIT_LR/(NUM_EPOCHS * 0.5))
model = TrafficSignNet.build(width = 32, height = 32, depth = 3, classes=numLabels)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metric = ["accuracy"])


[INFO] compiling model...


NameError: name 'numLabels' is not defined

In [24]:
print("[INFO] training network...")
H = model.fit(aug.flow(trainX, trainY, batch_size = BS), validation_data = (testX, testY), steps_per_epoch = trainX.shape[0] // BS, epochs = NUM_EPOCHS, class_weight = classWeight, verbose = 1)

[INFO] training network...


NameError: name 'model' is not defined

In [25]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))
print("[INFO] serializing network to '{}'...".format(args["model"]))
model.save(args["model"])


[INFO] evaluating network...


NameError: name 'model' is not defined

In [26]:
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

NameError: name 'H' is not defined

<Figure size 432x288 with 0 Axes>